In [16]:
import pandas as pd
import numpy as np
from pymoo.optimize import minimize

#%matplotlib qt
from pymoo.util.termination.f_tol import MultiObjectiveSpaceToleranceTermination
import PlotUtil
from notebooks.optimization_problems.constraints import Requirements
%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
columns = ['strand_name', 'tof',
           'r_a_x', 'r_a_y', 'r_a_z',
           'v_a_x', 'v_a_y', 'v_a_z',
           'r_b_x', 'r_b_y', 'r_b_z',
           'd',
           'r_ab_sff_x', 'r_ab_sff_y', 'r_ab_sff_z']

file_path = "example_data.h5"

row_limit = -1

store_500km = pd.HDFStore(file_path)

instances_500km_df = store_500km.select('contact_instances', 'columns = %s' % str(columns), stop = row_limit)

# Sort by pass id
instances_500km_df = instances_500km_df.sort_index(0)

In [18]:
N_passes = 20

instances_df = instances_500km_df.loc[0:N_passes,:]

In [ ]:
from notebooks.optimization_problems.vcm_problem import VCMProblem
#from notebooks.optimization_problems.vcm_problem_parallel import VCMProblemPar
from notebooks.optimization_problems.design_vector import design_vector_default_scm, SystemParameters

import multiprocessing

# the number of processes to be used
# n_proccess = 8
# pool = multiprocessing.Pool(n_proccess)

sys_param = SystemParameters()
sys_param.margin_dB = 3.0
sys_param.Gtx_dBi_bounds = (20., 20.)
sys_param.Ptx_dBm_bounds = (20., 60.)

modcods_df = pd.read_pickle('dvbs2.pkl')
sys_param.EsN0_req_dB_list = modcods_df['isend'][-1:].to_numpy()
sys_param.eta_bitsym_list = modcods_df['eta'][-1:].to_numpy()
sys_param.eta_maee_list = modcods_df['maee_12'][-1:].to_numpy()
sys_param.EsN0_req_dB_list = modcods_df['isend'].to_numpy()
sys_param.eta_bitsym_list = modcods_df['eta'].to_numpy()
sys_param.eta_maee_list = modcods_df['maee_12'].to_numpy()

requirements = Requirements()
requirements.min_throughput = 5e9
#requirements.max_throughput = 5e9

min_power = False
problem = VCMProblem(instances_df, sys_param, requirements=requirements, min_power=min_power)
#problem = VCMProblem(instances_df, sys_param, parallelization = ('starmap', pool.starmap))
#problem = VCMProblemPar(instances_df, sys_param)

sampling, crossover, mutation = design_vector_default_scm(problem.x_length, problem.x_indices)

converge_algo_settings = {'pop_size': 200, 'n_offsprings': 100}
unconverge_algo_settings = {'pop_size': 100, 'n_offsprings': 20}

converge_term_settings = {'f_tol': 0.001, 'n_max_gen': 2000, 'n_max_evals': 200000}
unconverge_term_settings =  {'f_tol': 0.005, 'n_max_gen': 100, 'n_max_evals': 2000}

converge = True
if converge:
    algo_settings = converge_algo_settings
    term_settings = converge_term_settings
    postfix = '_converged'
else:
    algo_settings = unconverge_algo_settings
    term_settings = unconverge_term_settings
    postfix = '_unconverged'

from pymoo.util.termination.default import MultiObjectiveDefaultTermination

termination = MultiObjectiveDefaultTermination(
    #x_tol=1e-8,
    #cv_tol=1e-6,
    f_tol=term_settings['f_tol'], #f_tol=0.005,
    nth_gen=100,
    n_last=30,
    n_max_gen=term_settings['n_max_gen'], #n_max_gen=1000,
    n_max_evals=term_settings['n_max_evals'], #n_max_evals=100000
)

# termination = MultiObjectiveSpaceToleranceTermination(tol=0.0001,
#                                                       n_last=30,
#                                                       nth_gen=5,
#                                                       n_max_gen=None,
#                                                       n_max_evals=None)

from pymoo.algorithms.nsga2 import NSGA2

algorithm = NSGA2(
    pop_size=algo_settings['pop_size'],
    n_offsprings=algo_settings['n_offsprings'],
    sampling=sampling,
    crossover=crossover,
    mutation=mutation,
    eliminate_duplicates=True,
)

res = minimize(problem,
               algorithm,
               termination=termination,
               seed=1,
               save_history=True,
               verbose=True
               )

#res = minimize(problem, GA())
# pool.close()
# problem.pool.close()

print('Processes:', res.exec_time)
print("Best solution found: %s" % res.X)

n_gen |  n_eval |   cv (min)   |   cv (avg)   |  n_nds  |     eps      |  indicator  
    1 |     200 |  0.00000E+00 |  1.84490E+02 |       5 |            - |            -
    2 |     300 |  0.00000E+00 |  1.05605E+02 |       5 |  0.095241885 |            f
    3 |     400 |  0.00000E+00 |  5.85350E+01 |       6 |  0.007398905 |            f
    4 |     500 |  0.00000E+00 |  1.81050E+01 |       8 |  0.031016331 |            f
    5 |     600 |  0.00000E+00 |  0.00000E+00 |      10 |  0.017519554 |        ideal
    6 |     700 |  0.00000E+00 |  0.00000E+00 |      12 |  0.026695730 |        ideal
    7 |     800 |  0.00000E+00 |  0.00000E+00 |      12 |  0.046605790 |        nadir
    8 |     900 |  0.00000E+00 |  0.00000E+00 |      10 |  0.110559101 |        ideal
    9 |    1000 |  0.00000E+00 |  0.00000E+00 |      11 |  0.002706085 |        ideal
   10 |    1100 |  0.00000E+00 |  0.00000E+00 |      13 |  0.018975092 |            f
   11 |    1200 |  0.00000E+00 |  0.00000E+00 |      1

C:\git\hermes-optimization\cenv\lib\site-packages\pymoo\operators\mutation\polynomial_mutation.py:34: RuntimeWarning: invalid value encountered in true_divide
  delta1 = (X - xl) / (xu - xl)
C:\git\hermes-optimization\cenv\lib\site-packages\pymoo\operators\mutation\polynomial_mutation.py:35: RuntimeWarning: invalid value encountered in true_divide
  delta2 = (xu - X) / (xu - xl)


In [ ]:
x_pass = res.X[:, problem.x_indices['pass']].astype('bool')
x_Ptx_dBm = res.X[:, problem.x_indices['power'][0]].astype('float64')
x_modcod = np.squeeze(res.X[:, problem.x_indices['modcod']].astype('int64'))

f_energy = res.F[:,1] / 1e3 # Kilo Joule
f_throughput = ((res.F[:,0] * -1) / 1e9)   # Gigabit

In [ ]:
import colorcet as cc

from matplotlib.lines import Line2D
import numpy as np
import matplotlib.pyplot as plt
#from PlotUtil import apply_report_formatting, apply_report_formatting_single
import matplotlib.colors as mcol
import matplotlib.cm as cm

isend_req = 10.69

fig = plt.figure(figsize=(3.2, 2.4))

ax = fig.add_subplot(1,1,1)
ax.grid()
ax.set_axisbelow(True)

cm1 = mcol.LinearSegmentedColormap.from_list("BlueRed",["c","r"])
#cm1 = mcol.LinearSegmentedColormap.from_list("BlueRed",["#4a569d","#dc2424"])
#cm1 = cc.cm.linear_kryw_5_100_c67_r
#cm1 = cc.cm.CET_L18

min_Ptx = np.nanmin(x_Ptx_dBm)
max_Ptx = np.nanmax(x_Ptx_dBm)

cnorm = mcol.Normalize(vmin=min_Ptx,vmax=max_Ptx)

cpick = cm.ScalarMappable(norm=cnorm,cmap=cm1)
cpick.set_array([])

cp = cpick.to_rgba(x_Ptx_dBm.tolist())
ax.scatter(f_energy, f_throughput, c=cp, marker='.', s=1)

ax.set_xlabel("Energy used [kJ]")
ax.set_ylabel("Throughput [Gbit]")
cb = plt.colorbar(cpick,label="Power [dBm]")
cb.ax.tick_params(labelsize=8)
cb.ax.yaxis.label.set_size(8)
#ax.set_ylim((0, 80))
#ax.set_xscale("log")

ax.text(np.max(f_energy), np.max(f_throughput),
        "(%.2f, %.2f)" % (np.max(f_energy), np.max(f_throughput)),
        horizontalalignment='right')
# for modcod in range(np.max(m)):
#     m_throughput = f_throughput[m == modcod] # throughputs with this modcod
#     if len(m_throughput) > 0:
#         max_throughput = np.max(m_throughput) # maximum throughput with this modcod
#         ax.axhline(y=max_throughput, color='r', linestyle='-')
#         ax.text(np.max(f_energy), max_throughput, '%d' % modcod)

PlotUtil.apply_report_formatting(4.72, 2.76)
plt.tight_layout()

fig.savefig('D:/git/thesis_report_ae/figures/vcm_pareto%s.pdf' % postfix)
#plt.savefig('D:/git/thesis_report_ae/figures/link_budget_pareto%s.svg' % postfix)
plt.show()
plt.close()

In [ ]:
fig = plt.figure(figsize=(3.2, 2.4))

ax = fig.add_subplot(1,1,1)
ax.grid()
ax.set_axisbelow(True)

ax.scatter(x_modcod, f_throughput, marker='.', s=1)
ax.set_xlabel("Modcod")
ax.set_ylabel("Throughput [Gbit]")
# ax.set_xlim((5.5, 11))
# ax.set_ylim((0, 80))

PlotUtil.apply_report_formatting(3.146, 2.76)
plt.tight_layout()

fig.savefig('D:/git/thesis_report_ae/figures/vcm_modcod_vs_throughput.pdf')

In [ ]:
fig = plt.figure(figsize=(3.2, 2.4))

import matplotlib.pyplot as plt
from pymoo.performance_indicator.hv import Hypervolume

# create the performance indicator object with reference point (4,4)
metric = Hypervolume(ref_point=np.array([0,1e18]))

# collect the population in each generation
pop_each_gen = [a.pop for a in res.history]

# receive the population in each generation
obj_and_feasible_each_gen = [pop[pop.get("feasible")[:,0]].get("F") for pop in pop_each_gen]

# calculate for each generation the HV metric
hv = [metric.calc(f) for f in obj_and_feasible_each_gen]

# function evaluations at each snapshot
n_evals = np.array([a.evaluator.n_eval for a in res.history])

# visualze the convergence curve
plt.plot(n_evals, hv, '-o', markersize=3)
plt.title("Convergence")
plt.xlabel("Function Evaluations")
plt.ylabel("Hypervolume")
PlotUtil.apply_report_formatting()
plt.tight_layout()
plt.grid()
#plt.savefig('D:/git/thesis_report_ae/figures/link_budget_hypervolume%s.svg' % postfix)
fig.savefig('D:/git/thesis_report_ae/figures/visibility_hypervolume%s.pdf' % postfix)
plt.show()
plt.close()
